In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [4]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [7]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

2022-06-22 22:23:56.125923: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-22 22:23:56.126342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [1]:
import os

import cv2
import numpy as np
import tensorflow as tf
from moviepy.editor import *


def main():

    # Dictionary that maps from joint names to keypoint indices.
    KEYPOINT_DICT = {
        "nose": 0,
        "left_eye": 1,
        "right_eye": 2,
        "left_ear": 3,
        "right_ear": 4,
        "left_shoulder": 5,
        "right_shoulder": 6,
        "left_elbow": 7,
        "right_elbow": 8,
        "left_wrist": 9,
        "right_wrist": 10,
        "left_hip": 11,
        "right_hip": 12,
        "left_knee": 13,
        "right_knee": 14,
        "left_ankle": 15,
        "right_ankle": 16,
    }

    # Maps bones to a matplotlib color name.
    KEYPOINT_EDGE_INDS_TO_COLOR = {
        (0, 1): (255, 0, 255),
        (0, 2): (0, 255, 255),
        (1, 3): (255, 0, 255),
        (2, 4): (0, 255, 255),
        (0, 5): (255, 0, 255),
        (0, 6): (0, 255, 255),
        (5, 7): (255, 0, 255),
        (7, 9): (255, 0, 255),
        (6, 8): (0, 255, 255),
        (8, 10): (0, 255, 255),
        (5, 6): (255, 255, 0),
        (5, 11): (255, 0, 255),
        (6, 12): (0, 255, 255),
        (11, 12): (255, 255, 0),
        (11, 13): (255, 0, 255),
        (13, 15): (255, 0, 255),
        (12, 14): (0, 255, 255),
        (14, 16): (0, 255, 255),
    }

    # Load model
    model = tf.lite.Interpreter(
        model_path="lite-model_movenet_singlepose_lightning_3.tflite"
    )
    model.allocate_tensors()

    # Define path for output video file.
    output_file_path = "result.avi"

    # Define path for input video file.
    input_file_path = "demo2.mov"

    # Get video input from primary camera
    cap = cv2.VideoCapture(input_file_path)

    # Check if camera opened successfully
    if cap.isOpened() == False:
        print("Error opening video stream or file")

    # Get dimesions of frame
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_file_path, fourcc, 20.0, (frame_width, frame_height))

    # Read until video is completed
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            img = frame.copy()
            # Resize the frame
            img = tf.image.resize_with_pad(
                np.expand_dims(img, axis=0), target_height=192, target_width=192
            )
            # Type convert to tf.float32
            img = tf.cast(img, dtype=tf.float32)
            # Set up input and output formats for image
            input_details = model.get_input_details()
            output_details = model.get_output_details()
            # Pass the frame in the desired input format
            model.set_tensor(input_details[0]["index"], np.array(img))
            # Make the predictions
            model.invoke()
            # Get the keypoints with confindence scores in the desired output format
            keypoints_with_scores = model.get_tensor(output_details[0]["index"])
            # Draw the edges
            draw_connections(
                frame, keypoints_with_scores, KEYPOINT_EDGE_INDS_TO_COLOR, 0.4
            )
            # Draw the keypoints
            draw_keypoints(frame, keypoints_with_scores, confidence_threshold=0.4)
            # Display the resulting frame
            cv2.imshow("Pose Estimation", frame)
            # Write the frame to a video file
            out.write(frame)
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord("q"):
                break
        # Break the loop
        else:
            break

    # Release video capture object
    cap.release()
    # Close all windows
    cv2.destroyAllWindows()
    # Release video writer
    out.release()
    print("result.avi")
    # Convert the file to .mp4 and remove .avi file
    convert_to_mp4(output_file_path)


def draw_keypoints(frame, keypoints, confidence_threshold=0.5):
    """
    Method to draw the appropriate keypoints for each frame
    """
    # Get the dimensions of the frame
    y, x, _ = frame.shape
    # Normalizing the Keypoint Coordinates according to the image size
    new_keypoints = np.squeeze(np.multiply(keypoints, np.array([y, x, 1])))
    print(new_keypoints)
    for keypoint in new_keypoints:
        # Getting the coordinates and the confidence coordinates for each keypoint
        ky, kx, kp_conf = keypoint
        # if the confidence score is less than the threshold, just ignore the keypoint detection altogether.
        if kp_conf > confidence_threshold:
            # Draw a circle filled with green color at the keypoint location with a radius of 3
            cv2.circle(frame, (int(kx), int(ky)), 3, (0, 255, 0), -1)


def draw_connections(frame, keypoints, edges, confidence_threshold=0.5):
    """
    Method to draw the edges between the appropriate keypoints for each frame
    """
    # Get the dimensions of the frame
    y, x, _ = frame.shape
    # Normalizing the Keypoint Coordinates according to the image size
    new_keypoints = np.squeeze(np.multiply(keypoints, np.array([y, x, 1])))

    for vertices, edge_color in edges.items():
        # Grab the vertices for a particular edge
        v1, v2 = vertices
        # Get the coordinates and confidence score for first vertex
        y1, x1, c1 = new_keypoints[v1]
        # Get the coordinates and confidence score for second vertex
        y2, x2, c2 = new_keypoints[v2]
        # Check if the confidence score of both vertices is above the required threshold
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            # Draw line of width 2 from the first vertex to the second vertex
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), edge_color, 2)


def convert_to_mp4(out_file_path: str) -> None:
    """
    Method that takes the .avi file and converts it to mp4 file with the same name.
    """
    nw_out_file_path = out_file_path.replace(".avi", ".mp4")
    clip = VideoFileClip(out_file_path)
    clip.write_videofile(nw_out_file_path)
    print("Removing the .avi file....")
    os.remove(out_file_path)


if __name__ == "__main__":
    main()

Metal device set to: Apple M1 Pro
[[2.72479230e+02 8.05905007e+02 4.46377009e-01]
 [2.56563469e+02 8.23402848e+02 6.68262124e-01]
 [2.57730471e+02 7.82592062e+02 4.87371355e-01]
 [2.58040330e+02 8.54110225e+02 7.64895380e-01]
 [2.59167084e+02 7.51263614e+02 6.77757025e-01]
 [3.39091301e+02 9.34258300e+02 7.66062379e-01]
 [3.43929770e+02 6.85148728e+02 8.00840020e-01]
 [4.56615250e+02 9.81381762e+02 6.72080517e-01]
 [4.70905062e+02 6.55368998e+02 7.31982112e-01]
 [5.64096429e+02 9.43396983e+02 5.05636632e-01]
 [5.66824493e+02 6.80401765e+02 4.89425927e-01]
 [5.79632535e+02 8.83746178e+02 7.79677629e-01]
 [5.86390479e+02 7.33993170e+02 7.14443505e-01]
 [7.78869338e+02 9.05620869e+02 7.28744030e-01]
 [7.61968203e+02 7.36228475e+02 8.01589549e-01]
 [9.03429451e+02 9.47400534e+02 1.46770999e-01]
 [8.96966143e+02 7.32802350e+02 1.97818503e-01]]
[[2.71617404e+02 8.05711936e+02 4.10845041e-01]
 [2.55395034e+02 8.22469214e+02 6.60632670e-01]
 [2.56955227e+02 7.81550764e+02 4.84540135e-01]
 [2.5

2022-06-22 19:44:57.838937: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-22 19:44:57.839057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[[2.70954298e+02 8.06343049e+02 3.91888201e-01]
 [2.55006237e+02 8.23459432e+02 6.53022647e-01]
 [2.56562889e+02 7.83324611e+02 5.26873291e-01]
 [2.56100256e+02 8.54300158e+02 7.72901237e-01]
 [2.58498698e+02 7.52922603e+02 7.00107396e-01]
 [3.38346055e+02 9.35190389e+02 7.52194583e-01]
 [3.41983066e+02 6.92080833e+02 8.18574309e-01]
 [4.59198411e+02 9.80247188e+02 7.13226557e-01]
 [4.63768637e+02 6.59597871e+02 7.72424877e-01]
 [5.66097465e+02 9.38589574e+02 5.30970871e-01]
 [5.64572597e+02 6.79130945e+02 4.35495079e-01]
 [5.77077377e+02 8.90758331e+02 7.63224065e-01]
 [5.85021200e+02 7.38871116e+02 7.14673162e-01]
 [7.76915874e+02 9.13753402e+02 7.22128093e-01]
 [7.53154378e+02 7.28494830e+02 5.80413997e-01]
 [8.99535785e+02 9.45044868e+02 2.04649329e-01]
 [8.93113160e+02 7.40328537e+02 2.10428610e-01]]
[[2.70175771e+02 8.06664013e+02 3.78251672e-01]
 [2.53607701e+02 8.23080436e+02 6.30742252e-01]
 [2.55165930e+02 7.83253157e+02 5.12590706e-01]
 [2.55125295e+02 8.54670657e+02 7.68045

[[2.73520110e+02 8.08386201e+02 5.23984790e-01]
 [2.56890757e+02 8.25499156e+02 7.13158727e-01]
 [2.57292815e+02 7.84701405e+02 5.78194201e-01]
 [2.57117704e+02 8.57853935e+02 8.30579698e-01]
 [2.57689739e+02 7.54344538e+02 7.48499513e-01]
 [3.38496784e+02 9.32180532e+02 7.23470747e-01]
 [3.43063245e+02 6.90856892e+02 9.00365889e-01]
 [4.15095170e+02 1.06180127e+03 5.71478963e-01]
 [4.45365036e+02 5.52615132e+02 6.70921445e-01]
 [4.77765326e+02 1.20804260e+03 6.35394454e-01]
 [5.28957775e+02 4.57468659e+02 4.07895327e-01]
 [5.86906171e+02 8.96367763e+02 8.61593902e-01]
 [5.84771626e+02 7.45615219e+02 8.12356830e-01]
 [8.01087556e+02 9.15015435e+02 7.40796149e-01]
 [7.80879965e+02 7.33273029e+02 7.95097470e-01]
 [9.07087770e+02 9.33123146e+02 8.08487907e-02]
 [8.88289819e+02 7.22728488e+02 1.64275929e-01]]
[[2.74094188e+02 8.08141953e+02 5.18346012e-01]
 [2.57886077e+02 8.26922539e+02 7.00778365e-01]
 [2.58129036e+02 7.86314238e+02 5.81296206e-01]
 [2.58017832e+02 8.57927610e+02 8.24260

[[2.78550024e+02 8.04735768e+02 6.78708076e-01]
 [2.62063644e+02 8.25236514e+02 7.99547374e-01]
 [2.60894871e+02 7.81763388e+02 5.56153774e-01]
 [2.63537496e+02 8.61020411e+02 6.40900254e-01]
 [2.61430696e+02 7.50158007e+02 7.00610936e-01]
 [3.36017683e+02 9.45530466e+02 7.91830063e-01]
 [3.43234348e+02 6.71071991e+02 8.82425368e-01]
 [3.51703906e+02 1.12553355e+03 6.52029157e-01]
 [3.71238155e+02 4.60285252e+02 5.30511975e-01]
 [3.15569690e+02 1.30169809e+03 6.56792402e-01]
 [3.70112851e+02 3.10945664e+02 4.26127285e-01]
 [6.00317452e+02 8.81903436e+02 8.86492133e-01]
 [5.97707255e+02 7.37186539e+02 8.86040807e-01]
 [8.04598074e+02 8.92097611e+02 6.39289796e-01]
 [7.85082364e+02 7.19202665e+02 7.30769038e-01]
 [9.17549801e+02 8.98469768e+02 1.24544576e-01]
 [9.12208257e+02 7.17941018e+02 2.68015593e-01]]
[[2.79763455e+02 8.00782766e+02 7.12770641e-01]
 [2.63129758e+02 8.23062186e+02 7.68718183e-01]
 [2.61979669e+02 7.78220088e+02 5.08902490e-01]
 [2.64155509e+02 8.56573942e+02 7.90258

...

[[2.83410637e+02 8.20112582e+02 5.59345782e-01]
 [2.69642311e+02 8.41057985e+02 4.47362781e-01]
 [2.68911999e+02 8.07265000e+02 5.79624414e-01]
 [2.84171752e+02 8.63368642e+02 6.70161664e-01]
 [2.79641662e+02 7.70646152e+02 6.92800999e-01]
 [3.72871556e+02 9.23097564e+02 8.28441024e-01]
 [3.42056065e+02 6.92451090e+02 7.52382517e-01]
 [4.88174636e+02 9.64641623e+02 7.51062036e-01]
 [3.23077193e+02 5.51854291e+02 5.85467815e-01]
 [5.96319180e+02 9.41142608e+02 4.96140808e-01]
 [1.81093848e+02 5.80287160e+02 3.61257136e-01]
 [6.06381712e+02 8.70783545e+02 8.29663813e-01]
 [6.10311556e+02 7.33879182e+02 8.35674822e-01]
 [8.20610604e+02 9.05891429e+02 7.68111885e-01]
 [8.10810521e+02 7.10169570e+02 6.96415782e-01]
 [8.99541578e+02 9.26722698e+02 9.14061069e-02]
 [9.03806098e+02 6.86788116e+02 1.33360684e-01]]
[[2.85817480e+02 8.23819020e+02 5.96170962e-01]
 [2.72225375e+02 8.43666251e+02 5.06801128e-01]
 [2.69670716e+02 8.08408120e+02 3.97140712e-01]
 [2.85419687e+02 8.63128209e+02 6.68705

[[2.80373261e+02 8.14410936e+02 5.45228124e-01]
 [2.67117682e+02 8.35622939e+02 4.94651258e-01]
 [2.67260381e+02 8.00498350e+02 6.39816821e-01]
 [2.75729359e+02 8.60136795e+02 7.72723556e-01]
 [2.72252315e+02 7.65909666e+02 6.42850161e-01]
 [3.62706735e+02 9.18327523e+02 8.06794107e-01]
 [3.38238262e+02 6.85715098e+02 6.75467789e-01]
 [4.77236663e+02 9.71303458e+02 7.40779042e-01]
 [3.15632036e+02 5.32297897e+02 6.58563316e-01]
 [5.92680559e+02 9.55620743e+02 5.91326892e-01]
 [1.78427871e+02 5.24693835e+02 4.88688082e-01]
 [5.99721036e+02 8.70456015e+02 8.43602598e-01]
 [5.99336472e+02 7.26997384e+02 8.68077993e-01]
 [8.14291620e+02 9.02939025e+02 7.85002530e-01]
 [7.99436259e+02 6.99605572e+02 6.35199249e-01]
 [9.20745471e+02 9.36255827e+02 1.76897213e-01]
 [9.26412871e+02 6.83134400e+02 2.48217151e-01]]
[[2.81371236e+02 8.21014062e+02 6.12512052e-01]
 [2.65986857e+02 8.42355938e+02 3.98401350e-01]
 [2.65898328e+02 8.06340201e+02 6.41173124e-01]
 [2.76441679e+02 8.61219227e+02 7.65930

[[2.86609526e+02 7.67241222e+02 4.28262144e-01]
 [2.67131973e+02 7.83693854e+02 4.00520593e-01]
 [2.72515730e+02 7.55280104e+02 4.30855483e-01]
 [2.70124755e+02 8.37514057e+02 3.76279056e-01]
 [2.74177036e+02 7.51987810e+02 4.73153114e-01]
 [3.39811667e+02 9.01537077e+02 6.16262376e-01]
 [3.46537714e+02 6.90761105e+02 6.91478908e-01]
 [4.40866810e+02 9.45634750e+02 5.01809001e-01]
 [3.71014202e+02 6.36425902e+02 3.69494677e-01]
 [5.79776280e+02 9.41881192e+02 3.70519280e-01]
 [3.39500005e+02 7.21682555e+02 3.67966443e-01]
 [5.84049747e+02 8.60533848e+02 7.34364092e-01]
 [5.80816097e+02 7.05799141e+02 5.98453403e-01]
 [8.01084788e+02 8.88452103e+02 5.53924382e-01]
 [7.83834364e+02 7.00220076e+02 7.19412208e-01]
 [9.01118460e+02 9.46222411e+02 1.84066132e-01]
 [9.01471481e+02 6.82329576e+02 2.52075285e-01]]
[[2.84766172e+02 7.73960654e+02 3.55385274e-01]
 [2.68290237e+02 8.05015887e+02 4.25283313e-01]
 [2.70916028e+02 7.65993335e+02 4.07434374e-01]
 [2.76098764e+02 8.43209910e+02 5.23752

[[2.76673615e+02 8.04410942e+02 4.33052033e-01]
 [2.61921573e+02 8.31224363e+02 5.99089980e-01]
 [2.63182415e+02 7.91807992e+02 6.56878054e-01]
 [2.66948075e+02 8.58361645e+02 5.68316698e-01]
 [2.64174821e+02 7.62764529e+02 6.89487398e-01]
 [3.48765728e+02 9.01411743e+02 6.69183314e-01]
 [3.43459847e+02 6.95318280e+02 7.09507048e-01]
 [4.21760931e+02 9.43734652e+02 4.33843344e-01]
 [4.14424210e+02 6.61605536e+02 3.38478178e-01]
 [4.11551049e+02 9.48786550e+02 2.72357315e-01]
 [3.85992322e+02 7.95944023e+02 2.02191651e-01]
 [5.85906200e+02 8.72124467e+02 7.39098787e-01]
 [5.82792349e+02 7.19277982e+02 7.39693344e-01]
 [7.92287829e+02 8.92562642e+02 6.58850253e-01]
 [7.88761475e+02 6.97560827e+02 7.13120818e-01]
 [8.86199627e+02 9.46616857e+02 2.42150366e-01]
 [8.98860962e+02 6.82707607e+02 2.57988155e-01]]
[[2.78667312e+02 8.09568862e+02 5.04863262e-01]
 [2.61980844e+02 8.24956877e+02 5.57180047e-01]
 [2.64831040e+02 7.86469168e+02 6.48606002e-01]
 [2.66859659e+02 8.57516073e+02 5.94276

NameError: name 'func' is not defined